# СБОР СПИСКА СЛОВ

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor

# Функция для замены специальных символов
def sanitize_word(word):
    sanitized = word.replace(' ', '-').replace('/', '-').replace(',','')
    return sanitized

# Функция для парсинга страницы по слову и букве с повторными попытками
def parse_word_page(args):
    letter, word = args
    sanitized_word = sanitize_word(word)
    url = f'https://dictionary.cambridge.org/browse/english-russian/{letter}/{sanitized_word}/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    attempt = 0
    while True:  # Бесконечный цикл
        attempt += 1
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Установить тайм-аут
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                elements = soup.find_all('div', class_='hlh32')
                results = [element.text.replace('/', ' ').strip() for element in elements]
                return results
            else:
                print(f"Ошибка доступа к слову: {word}, статус: {response.status_code}, попытка {attempt}")
        except requests.exceptions.RequestException as e:
            print(f"Исключение при доступе к слову: {word}, ошибка: {e}, попытка {attempt}")
        
        time.sleep(1)  # Небольшая пауза перед повторной попыткой
        if attempt >= 10:  # Ограничение на количество попыток
            print(f"Не удалось получить доступ к слову: {word} после {attempt} попыток")
            return None

# Загрузка данных из файла Excel
df = pd.read_excel('cambridge_hlh32_data.xlsx')

# Список для хранения всех запросов
requests_to_process = []

# Проход по строкам файла
for index, row in df.iterrows():
    entry = row['Entry']
    first_word = entry.split('...')[0].strip()  # Извлечение первого слова
    letter = first_word[0].lower()  # Определение буквы
    requests_to_process.append((letter, first_word))

# Использование ThreadPoolExecutor для многопоточности
all_results = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(parse_word_page, args): args for args in requests_to_process}

    for future in tqdm(futures):
        word_data = future.result()
        if word_data:
            all_results.extend(word_data)

# Сохранение результатов в Excel
results_df = pd.DataFrame(all_results)
results_df.to_excel('parsed_cambridge_data.xlsx', index=False)

print("Данные успешно сохранены в файл parsed_cambridge_data.xlsx")


## Парсинг фразовых глаголов

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Функция для очистки фразы и замены символов
def sanitize_phrase(phrase):
    sanitized = phrase.replace(' ', '-')
    sanitized = sanitized.replace('/', '-').replace(',', '').replace('\'', '-').replace('.', '')
    return sanitized

# Функция для получения данных с Cambridge Dictionary с обработкой ошибок и повторными попытками
def parse_cambridge(word, retries=20, delay=0.1):
    base_url = 'https://dictionary.cambridge.org/ru/словарь/англо-русский/'
    word_url = sanitize_phrase(word)
    url = f'{base_url}{word_url}'
    
    # Заголовки, включая User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Увеличиваем timeout до 10 секунд
            response.raise_for_status()  # Проверяем, успешен ли запрос
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Поиск pv-block
            pv_block = soup.find('div', {'class': 'pv-block'})
            if not pv_block:
                return None

            # Извлечение слова
            headword = pv_block.find('b').text if pv_block.find('b') else 'Нет слова'

            # Извлечение переводов (trans) и примеров (eg)
            translations = []
            for sense_block in pv_block.find_all('div', {'class': 'dsense'}):
                trans = sense_block.find('span', {'class': 'trans'}).text if sense_block.find('span', {'class': 'trans'}) else 'Нет перевода'
                eg = sense_block.find('span', {'class': 'eg'}).text if sense_block.find('span', {'class': 'eg'}) else 'Нет примера'
                translations.append({'translation': trans, 'example': eg})

            return {'word': word, 'headword': headword, 'translations': translations}
        
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f'Попытка {attempt+1} для слова "{word}" не удалась. Ошибка: {e}')
            if attempt < retries - 1:
                time.sleep(delay)  # Ожидание перед повторной попыткой
            else:
                print(f'Не удалось подключиться к {url} после {retries} попыток.')
                return None

# Функция для обработки каждой фразы
def process_phrase(phrase):
    data = parse_cambridge(phrase)
    if data:
        results = []
        for translation in data['translations']:
            results.append({
                'word': data['word'],
                'headword': data['headword'],
                'translation': translation['translation'],
                'example': translation['example']
            })
        return results
    return []

# Чтение фраз из Excel файла
file_path = 'parsed_cambridge_data.xlsx'
parsed_data = pd.read_excel(file_path)

# Использование ThreadPoolExecutor для многопоточного парсинга
results = []
with ThreadPoolExecutor(max_workers=10) as executor:  # Указываем количество потоков
    futures = {executor.submit(process_phrase, phrase): phrase for phrase in parsed_data[parsed_data.columns[0]]}
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг слов"):
        result = future.result()
        if result:
            results.extend(result)

# Преобразование результатов в DataFrame для удобного вывода
parsed_df = pd.DataFrame(results)

# Сохранение результатов в Excel, где каждый перевод на новой строке
parsed_df.to_excel('parsed_cambridge_pv.xlsx', index=False)


In [ ]:
import os

In [ ]:
os.startfile('parsed_cambridge_pv.xlsx')

# Перевод фразовых глаголов

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Загрузка файла
file_path = 'parsed_cambridge_pv.xlsx'
df = pd.read_excel(file_path)

# Функция для перевода текста
def translate_example(example):
    if example == 'Нет примера':
        return 'Нет примера'
    try:
        translation = GoogleTranslator(source='auto', target='ru').translate(example)
        return translation
    except Exception as e:
        print(f"Ошибка при переводе: {e}")
        return 'Ошибка'

# Функция для многопоточного перевода
def translate_examples_in_parallel(examples):
    translations = [None] * len(examples)  # Инициализируем список переводов
    with ThreadPoolExecutor(max_workers=5) as executor:  # Укажите количество потоков
        future_to_index = {executor.submit(translate_example, example): index for index, example in enumerate(examples)}
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Перевод примеров"):
            index = future_to_index[future]
            translations[index] = future.result()  # Сохраняем результат в правильный индекс
    return translations

# Применение многопоточного перевода ко всем примерам
df['translated_example'] = translate_examples_in_parallel(df['example'].tolist())

# Сохранение результата в новый файл
output_path = 'parsed_cambridge_pv_translated.xlsx'
df.to_excel(output_path, index=False)

print(f"Переводы записаны в файл: {output_path}")


# Парсинг идиом

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Функция для очистки фразы и замены символов
def sanitize_phrase(phrase):
    sanitized = phrase.replace(' ', '-')
    sanitized = sanitized.replace('/', '-').replace(',', '').replace('\'', '-').replace('.', '')
    return sanitized

# Функция для получения данных с Cambridge Dictionary с обработкой ошибок и повторными попытками
def parse_cambridge(word, retries=20, delay=0.1):
    base_url = 'https://dictionary.cambridge.org/ru/словарь/англо-русский/'
    word_url = sanitize_phrase(word)
    url = f'{base_url}{word_url}'
    
    # Заголовки, включая User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Увеличиваем timeout до 10 секунд
            response.raise_for_status()  # Проверяем, успешен ли запрос
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Поиск idiom-block
            idiom_block = soup.find('div', {'class': 'idiom-block'})
            if not idiom_block:
                return None

            # Извлечение headword идиомы
            headword = idiom_block.find('h2', {'class': 'headword'}).text if idiom_block.find('h2', {'class': 'headword'}) else 'Нет слова'

            # Извлечение переводов (trans) и примеров (eg)
            translations = []
            for idiom in idiom_block.find_all('div', {'class': 'def-block'}):
                trans = idiom.find('span', {'class': 'trans'}).text if idiom.find('span', {'class': 'trans'}) else 'Нет перевода'
                example = idiom.find('span', {'class': 'eg'}).text if idiom.find('span', {'class': 'eg'}) else 'Нет примера'
                translations.append({'translation': trans, 'example': example})

            return {'word': word, 'headword': headword, 'translations': translations}
        
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f'Попытка {attempt+1} для слова "{word}" не удалась. Ошибка: {e}')
            if attempt < retries - 1:
                time.sleep(delay)  # Ожидание перед повторной попыткой
            else:
                print(f'Не удалось подключиться к {url} после {retries} попыток.')
                return None

# Функция для многопоточной обработки фраз
def process_words(word_list, max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_cambridge, word): word for word in word_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг слов"):
            word = futures[future]
            try:
                data = future.result()
                if data:
                    for translation in data['translations']:
                        results.append({
                            'word': data['word'],
                            'headword': data['headword'],
                            'translation': translation['translation'],
                            'example': translation['example']
                        })
            except Exception as e:
                print(f"Ошибка при обработке слова {word}: {e}")
    return results

# Чтение фраз из Excel файла
file_path = 'parsed_cambridge_data.xlsx'
parsed_data = pd.read_excel(file_path)

# Многопоточная обработка фраз
word_list = parsed_data[parsed_data.columns[0]].tolist()  # Преобразуем столбец в список
results = process_words(word_list)

# Преобразование результатов в DataFrame для удобного вывода
parsed_df = pd.DataFrame(results)

# Сохранение результатов в Excel, где каждый перевод на новой строке
parsed_df.to_excel('parsed_cambridge_idioms.xlsx', index=False)


# Перевод идиом

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Загрузка файла
file_path = 'parsed_cambridge_idioms.xlsx'
df = pd.read_excel(file_path)

# Функция для перевода текста
def translate_example(example):
    if example == 'Нет примера':
        return 'Нет примера'
    try:
        translation = GoogleTranslator(source='auto', target='ru').translate(example)
        return translation
    except Exception as e:
        print(f"Ошибка при переводе: {e}")
        return 'Ошибка'

# Функция для многопоточного перевода
def translate_examples_in_parallel(examples):
    translations = [None] * len(examples)  # Инициализируем список переводов
    with ThreadPoolExecutor(max_workers=5) as executor:  # Укажите количество потоков
        future_to_index = {executor.submit(translate_example, example): index for index, example in enumerate(examples)}
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Перевод примеров"):
            index = future_to_index[future]
            translations[index] = future.result()  # Сохраняем результат в правильный индекс
    return translations

# Применение многопоточного перевода ко всем примерам
df['translated_example'] = translate_examples_in_parallel(df['example'].tolist())

# Сохранение результата в новый файл
output_path = 'parsed_cambridge_idioms_translated.xlsx'
df.to_excel(output_path, index=False)

print(f"Переводы записаны в файл: {output_path}")


# Парсинг phrase-block

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Функция для очистки фразы и замены символов
def sanitize_phrase(phrase):
    sanitized = phrase.replace(' ', '-')
    sanitized = sanitized.replace('/', '-').replace(',', '').replace('\'', '-').replace('.', '')
    return sanitized

# Функция для получения данных с Cambridge Dictionary с обработкой ошибок и повторными попытками
def parse_cambridge(word, retries=20, delay=0.1):
    base_url = 'https://dictionary.cambridge.org/ru/словарь/англо-русский/'
    word_url = sanitize_phrase(word)
    url = f'{base_url}{word_url}'
    
    # Заголовки, включая User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Увеличиваем timeout до 10 секунд
            response.raise_for_status()  # Проверяем, успешен ли запрос
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Поиск всех блоков dsense
            dsense_blocks = soup.find_all('div', {'class': 'dsense'})
            if not dsense_blocks:
                print(f'Блок dsense не найден для слова: {word}')
                return None
            
            translations = []  # Список для хранения переводов
            
            for dsense in dsense_blocks:
                # Поиск всех phrase-block в текущем dsense
                phrase_blocks = dsense.find_all('div', {'class': 'phrase-block'})
                for phrase_block in phrase_blocks:
                    # Извлечение phrase-title
                    phrase_title = phrase_block.find('span', {'class': 'phrase-title'}).text if phrase_block.find('span', {'class': 'phrase-title'}) else 'Нет заголовка'

                    # Извлечение переводов (trans)
                    for phrase in phrase_block.find_all('div', {'class': 'def-block'}):
                        trans = phrase.find('span', {'class': 'trans'}).text if phrase.find('span', {'class': 'trans'}) else 'Нет перевода'
                        example = phrase.find('span', {'class': 'eg'}).text if phrase.find('span', {'class': 'eg'}) else 'Нет примера'
                        level = phrase.find('span', {'class': 'epp-xref'}).text if phrase.find('span', {'class': 'epp-xref'}) else 'без уровня'
                        translations.append({'phrase_title': phrase_title, 'translation': trans, 'example': example, 'level': level})

            return {'word': word, 'translations': translations}
        
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f'Попытка {attempt+1} для слова "{word}" не удалась. Ошибка: {e}')
            if attempt < retries - 1:
                time.sleep(delay)  # Ожидание перед повторной попыткой
            else:
                print(f'Не удалось подключиться к {url} после {retries} попыток.')
                return None

# Функция для многопоточной обработки фраз
def process_words(word_list, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_cambridge, word): word for word in word_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг слов"):
            word = futures[future]
            try:
                data = future.result()
                if data:
                    for translation in data['translations']:
                        results.append({
                            'word': data['word'],
                            'phrase_title': translation['phrase_title'],
                            'translation': translation['translation'],
                            'example': translation['example'],
                            'level': translation['level']
                        })
            except Exception as e:
                print(f"Ошибка при обработке слова {word}: {e}")
    return results

# Чтение фраз из Excel файла
file_path = 'parsed_cambridge_data.xlsx'
parsed_data = pd.read_excel(file_path)

# Многопоточная обработка фраз
word_list = parsed_data[parsed_data.columns[0]].tolist()  # Преобразуем столбец в список
results = process_words(word_list)

# Преобразование результатов в DataFrame для удобного вывода
parsed_df = pd.DataFrame(results)

# Сохранение результатов в Excel, где каждый перевод на новой строке
parsed_df.to_excel('parsed_cambridge_phrases.xlsx', index=False)


In [ ]:
import os

In [ ]:
os.startfile('parsed_cambridge_phrases.xlsx')

# Перевод фраз

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Загрузка файла
file_path = 'parsed_cambridge_phrases.xlsx'
df = pd.read_excel(file_path)

# Функция для перевода текста
def translate_example(example):
    if example == 'Нет примера':
        return 'Нет примера'
    try:
        translation = GoogleTranslator(source='auto', target='ru').translate(example)
        return translation
    except Exception as e:
        print(f"Ошибка при переводе: {e}")
        return 'Ошибка'

# Функция для многопоточного перевода
def translate_examples_in_parallel(examples):
    translations = [None] * len(examples)  # Инициализируем список переводов
    with ThreadPoolExecutor(max_workers=5) as executor:  # Укажите количество потоков
        future_to_index = {executor.submit(translate_example, example): index for index, example in enumerate(examples)}
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Перевод примеров"):
            index = future_to_index[future]
            translations[index] = future.result()  # Сохраняем результат в правильный индекс
    return translations

# Применение многопоточного перевода ко всем примерам
df['translated_example'] = translate_examples_in_parallel(df['example'].tolist())

# Сохранение результата в новый файл
output_path = 'parsed_cambridge_phrases_translated.xlsx'
df.to_excel(output_path, index=False)

print(f"Переводы записаны в файл: {output_path}")


In [ ]:
import os
os.startfile('parsed_cambridge_phrases_translated.xlsx')

# Парсинг рунонов

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Функция для очистки фразы и замены символов
def sanitize_phrase(phrase):
    sanitized = phrase.replace(' ', '-')
    sanitized = sanitized.replace('/', '-').replace(',', '').replace('\'', '-').replace('.', '')
    return sanitized

# Функция для получения данных с Cambridge Dictionary с обработкой ошибок и повторными попытками
def parse_cambridge(word, retries=3, delay=2):
    base_url = 'https://dictionary.cambridge.org/ru/словарь/англо-русский/'
    word_url = sanitize_phrase(word)
    url = f'{base_url}{word_url}'
    
    # Заголовки, включая User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Увеличиваем timeout до 10 секунд
            response.raise_for_status()  # Проверяем, успешен ли запрос
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Поиск всех блоков runon
            runon_blocks = soup.find_all('div', {'class': 'runon'})
            if not runon_blocks:
                return None
            
            translations = []  # Список для хранения переводов
            
            for runon in runon_blocks:
                # Извлечение runon-title
                runon_title = runon.find('h3', {'class': 'runon-title'}).text.strip() if runon.find('h3', {'class': 'runon-title'}) else 'Нет заголовка'

                # Извлечение переводов (trans)
                trans = runon.find('span', {'class': 'trans'}).text.strip() if runon.find('span', {'class': 'trans'}) else 'Нет перевода'
                example = runon.find('span', {'class': 'eg'}).text.strip() if runon.find('span', {'class': 'eg'}) else 'Нет примера'
                
                translations.append({'runon_title': runon_title, 'translation': trans, 'example': example})

            return {'word': word, 'translations': translations}
        
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f'Попытка {attempt+1} для слова "{word}" не удалась. Ошибка: {e}')
            if attempt < retries - 1:
                time.sleep(delay)  # Ожидание перед повторной попыткой
            else:
                print(f'Не удалось подключиться к {url} после {retries} попыток.')
                return None

# Функция для многопоточной обработки фраз
def process_words(word_list, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_cambridge, word): word for word in word_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг слов"):
            word = futures[future]
            try:
                data = future.result()
                if data:
                    for translation in data['translations']:
                        results.append({
                            'word': data['word'],
                            'runon_title': translation['runon_title'],
                            'translation': translation['translation'],
                            'example': translation['example']
                        })
            except Exception as e:
                print(f"Ошибка при обработке слова {word}: {e}")
    return results

# Чтение фраз из Excel файла
file_path = 'parsed_cambridge_data.xlsx'
parsed_data = pd.read_excel(file_path)

# Многопоточная обработка фраз
word_list = parsed_data[parsed_data.columns[0]].tolist()  # Преобразуем столбец в список
results = process_words(word_list)

# Преобразование результатов в DataFrame для удобного вывода
parsed_df = pd.DataFrame(results)

# Сохранение результатов в Excel, где каждый перевод на новой строке
parsed_df.to_excel('parsed_cambridge_runon.xlsx', index=False)


In [ ]:
import os
os.startfile('parsed_cambridge_runon.xlsx')

# Парсинг слов

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Функция для очистки фразы и замены символов
def sanitize_phrase(phrase):
    sanitized = phrase.replace(' ', '-')
    sanitized = sanitized.replace('/', '-').replace(',', '').replace('\'', '-').replace('.', '')
    return sanitized

# Функция для получения данных с Cambridge Dictionary с обработкой ошибок и повторными попытками
def parse_cambridge(word, retries=30, delay=0.2):
    base_url = 'https://dictionary.cambridge.org/ru/словарь/англо-русский/'
    word_url = sanitize_phrase(word)
    url = f'{base_url}{word_url}'
    
    # Заголовки, включая User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)  # Увеличиваем timeout до 10 секунд
            response.raise_for_status()  # Проверяем, успешен ли запрос
            soup = BeautifulSoup(response.content, 'html.parser')

            # Проверка на наличие phrasal verb
            phrasal_verb = soup.find('span', {'title': 'Verb with an adverb or preposition, with meaning different from meaning of its parts.'})
            if phrasal_verb:
                return None
            
            # Поиск всех блоков dsense
            dsense_blocks = soup.find_all('div', class_='sense-body')
            translations = []

            # Извлечение данных из каждого подходящего блока
            for dsense in dsense_blocks:
                # Проверка на наличие phrase-block внутри dsense
                phrase_block = dsense.find('div', class_='phrase-block')
                if phrase_block:
                    continue  # Пропускаем этот блок

                # Извлечение headword (dsense_hw)
                headword_block = soup.find('span', {'class': 'hw dhw'})
                headword = headword_block.text.strip() if headword_block else 'Нет headword'

                # Извлечение перевода (trans)
                trans = dsense.find('span', {'class': 'trans'})
                trans_text = trans.text.strip() if trans else 'Нет перевода'

                # Извлечение примера (examp)
                examp = dsense.find('div', {'class': 'examp'})
                examp_text = examp.text.strip() if examp else 'Нет примера'

                # Извлечение уровня сложности (epp-xref)
                epp_xref = dsense.find('span', {'class': 'epp-xref'})
                epp_xref_text = epp_xref.text.strip() if epp_xref else 'без уровня'

                # Сохраняем данные
                translations.append({
                    'headword': headword,
                    'translation': trans_text,
                    'example': examp_text,
                    'level': epp_xref_text
                })

            if not translations:  # Если переводы не найдены, проверяем наличие блока pos-header
                pos_header = soup.find('div', class_='pos-header')
                if pos_header:
                    hw = pos_header.find('span', class_='hw')
                    dhw = pos_header.find('span', class_='dhw')

                    headword = hw.text.strip() if hw else dhw.text.strip() if dhw else 'Нет headword'
                    translations.append({
                        'headword': headword,
                        'translation': 'Нет перевода',  # Здесь можно указать, если это нужно
                        'example': 'Нет примера',
                        'level': 'без уровня'
                    })

            return {'word': word, 'translations': translations}
        
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f'Попытка {attempt+1} для слова "{word}" не удалась. Ошибка: {e}')
            if attempt < retries - 1:
                time.sleep(delay)  # Ожидание перед повторной попыткой
            else:
                print(f'Не удалось подключиться к {url} после {retries} попыток.')
                return None

# Функция для многопоточной обработки фраз
def process_words(word_list, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_cambridge, word): word for word in word_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="Парсинг слов"):
            word = futures[future]
            try:
                data = future.result()
                if data:
                    for translation in data['translations']:
                        results.append({
                            'word': data['word'],
                            'headword': translation['headword'],
                            'translation': translation['translation'],  # Перевод берется из dsense
                            'example': translation['example'],
                            'level': translation['level']
                        })
            except Exception as e:
                print(f"Ошибка при обработке слова {word}: {e}")
    return results

# Чтение фраз из Excel файла
file_path = 'parsed_cambridge_data.xlsx'
parsed_data = pd.read_excel(file_path)

# Многопоточная обработка фраз
word_list = parsed_data[parsed_data.columns[0]].tolist()  # Преобразуем столбец в список
results = process_words(word_list)

# Преобразование результатов в DataFrame для удобного вывода
parsed_df = pd.DataFrame(results)

# Сохранение результатов в Excel, где каждый перевод на новой строке
parsed_df.to_excel('parsed_cambridge_dsense.xlsx', index=False)


Парсинг слов:   2%|█                                                             | 296/17930 [00:54<1:02:44,  4.68it/s]

Попытка 1 для слова "administration" не удалась. Ошибка: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /ru/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C/%D0%B0%D0%BD%D0%B3%D0%BB%D0%BE-%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B9/administration (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001450D5CCF50>, 'Connection to dictionary.cambridge.org timed out. (connect timeout=10)'))


Парсинг слов:   3%|█▉                                                            | 544/17930 [01:39<1:08:48,  4.21it/s]

Попытка 1 для слова "already" не удалась. Ошибка: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /ru/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C/%D0%B0%D0%BD%D0%B3%D0%BB%D0%BE-%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B9/already (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001450D709BD0>, 'Connection to dictionary.cambridge.org timed out. (connect timeout=10)'))


Парсинг слов:   4%|██▊                                                             | 803/17930 [02:26<48:46,  5.85it/s]

...

Парсинг слов:  97%|████████████████████████████████████████████████████████████▎ | 17436/17930 [52:57<01:47,  4.58it/s]

Попытка 1 для слова "vocal" не удалась. Ошибка: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /ru/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C/%D0%B0%D0%BD%D0%B3%D0%BB%D0%BE-%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B9/vocal (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001450B0EF290>, 'Connection to dictionary.cambridge.org timed out. (connect timeout=10)'))


Парсинг слов:  99%|█████████████████████████████████████████████████████████████▎| 17718/17930 [53:49<00:37,  5.66it/s]

Попытка 1 для слова "whooping cough" не удалась. Ошибка: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /ru/%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C/%D0%B0%D0%BD%D0%B3%D0%BB%D0%BE-%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%B8%D0%B9/whooping-cough (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001450DBE0A50>, 'Connection to dictionary.cambridge.org timed out. (connect timeout=10)'))


Парсинг слов: 100%|██████████████████████████████████████████████████████████████| 17930/17930 [54:26<00:00,  5.49it/s]


In [38]:
import os
os.startfile('parsed_cambridge_dsense.xlsx')

# Перевод слов и рунонов

In [39]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Загрузка файла
file_path = 'parsed_cambridge_words.xlsx'
df = pd.read_excel(file_path)

# Функция для перевода текста
def translate_example(example):
    if example == 'Нет примера':
        return 'Нет примера'
    try:
        translation = GoogleTranslator(source='auto', target='ru').translate(example)
        return translation
    except Exception as e:
        print(f"Ошибка при переводе: {e}")
        return 'Ошибка'

# Функция для многопоточного перевода
def translate_examples_in_parallel(examples):
    translations = [None] * len(examples)  # Инициализируем список переводов
    with ThreadPoolExecutor(max_workers=5) as executor:  # Укажите количество потоков
        future_to_index = {executor.submit(translate_example, example): index for index, example in enumerate(examples)}
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="Перевод примеров"):
            index = future_to_index[future]
            translations[index] = future.result()  # Сохраняем результат в правильный индекс
    return translations

# Применение многопоточного перевода ко всем примерам
df['translated_example'] = translate_examples_in_parallel(df['example'].tolist())

# Сохранение результата в новый файл
output_path = 'parsed_cambridge_words_translated.xlsx'
df.to_excel(output_path, index=False)

print(f"Переводы записаны в файл: {output_path}")


Перевод примеров:  25%|██████████████▋                                            | 6554/26385 [12:21<20:56, 15.78it/s]

Ошибка при переводе: Response ended prematurelyОшибка при переводе: Response ended prematurely

Ошибка при переводе: Response ended prematurely


Перевод примеров:  57%|█████████████████████████████████                         | 15029/26385 [23:56<11:32, 16.40it/s]

Ошибка при переводе: Response ended prematurelyОшибка при переводе: ('Connection aborted.', ConnectionAbortedError(10053, 'Программа на вашем хост-компьютере разорвала установленное подключение', None, 10053, None))

Ошибка при переводе: ('Connection aborted.', ConnectionAbortedError(10053, 'Программа на вашем хост-компьютере разорвала установленное подключение', None, 10053, None))
Ошибка при переводе: Response ended prematurely
Ошибка при переводе: ('Connection aborted.', ConnectionAbortedError(10053, 'Программа на вашем хост-компьютере разорвала установленное подключение', None, 10053, None))


Перевод примеров: 100%|██████████████████████████████████████████████████████████| 26385/26385 [36:33<00:00, 12.03it/s]


Переводы записаны в файл: parsed_cambridge_words_translated.xlsx


In [40]:
import os
os.startfile('parsed_cambridge_words_translated.xlsx')

# Озвучка

In [ ]:
pip install edge-tts

In [43]:
import os
import pandas as pd
from tqdm import tqdm
import edge_tts
import nest_asyncio
import asyncio
import re
import time

# Устанавливаем возможность повторного использования asyncio.run в Jupyter и других средах
nest_asyncio.apply()

# Загрузка файла
input_path = 'parsed_cambridge_words_translated.xlsx'
output_folder = 'audio_files_words'
os.makedirs(output_folder, exist_ok=True)

# Чтение данных
df = pd.read_excel(input_path)
headwords = df.iloc[:, 1]  # Извлекаем первый столбец с headword

# Функция для очистки имени файла от недопустимых символов
def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', " ", filename)

# Инициализация Edge TTS
async def generate_audio(text, file_path):
    communicate = edge_tts.Communicate(text, voice="en-US-GuyNeural")
    await communicate.save(file_path)

# Функция с несколькими попытками генерации аудио
def try_generate_audio(word, audio_file, retries=3, delay=2):
    sanitized_word = sanitize_filename(word)
    success = False
    
    for attempt in range(retries):
        try:
            asyncio.run(generate_audio(sanitized_word, audio_file))
            success = True
            break
        except Exception as e:
            print(f"Error generating audio for '{sanitized_word}' (Attempt {attempt+1}/{retries}): {e}")
            time.sleep(delay)  # Ожидание перед повторной попыткой
    
    return success

# Обрабатываем озвучку для каждого слова
audio_links = []
for word in tqdm(headwords, desc="Generating Audio"):
    audio_file = f"{output_folder}/{sanitize_filename(word)}.mp3"
    
    if not os.path.exists(audio_file):
        # Пытаемся сгенерировать аудиофайл с несколькими попытками
        success = try_generate_audio(word, audio_file)
        if not success:
            audio_file = "Error: Audio not generated"
    
    # Записываем путь к аудиофайлу (или сообщение об ошибке)
    audio_links.append(audio_file)

# Добавляем ссылки на аудио в столбец F
df['audio_path'] = audio_links

# Сохранение обновленного файла
output_path = 'parsed_cambridge_words_translated_with_audio.xlsx'
df.to_excel(output_path, index=False)

print(f"All audio files generated and saved in {output_folder}. Updated file saved as {output_path}.")


Generating Audio:  10%|█████▉                                                   | 2757/26385 [19:15<2:27:44,  2.67it/s]

Error generating audio for 'knickers' (Attempt 1/3): Connection timeout to host wss://speech.platform.bing.com/consumer/speech/synthesize/readaloud/edge/v1?TrustedClientToken=6A5AA1D4EAFF4E9FB37E23D68491D6F4&ConnectionId=2a7f5c56c13a4fe2b2af6f8fb83e0d6a


Generating Audio:  36%|████████████████████▍                                    | 9488/26385 [59:10<1:43:46,  2.71it/s]

Error generating audio for 'con artist' (Attempt 1/3): No audio was received. Please verify that your parameters are correct.


Generating Audio:  54%|████████████████████████████▉                         | 14158/26385 [1:28:24<1:51:02,  1.84it/s]

Error generating audio for 'horse-drawn' (Attempt 1/3): [WinError 10054] Удаленный хост принудительно разорвал существующее подключение


Generating Audio:  75%|█████████████████████████████████████████▉              | 19761/26385 [2:04:42<39:19,  2.81it/s]

Error generating audio for 'rustle' (Attempt 1/3): Connection timeout to host wss://speech.platform.bing.com/consumer/speech/synthesize/readaloud/edge/v1?TrustedClientToken=6A5AA1D4EAFF4E9FB37E23D68491D6F4&ConnectionId=6c2bb5305bba4eb58c219e22bd7342fe


Generating Audio: 100%|████████████████████████████████████████████████████████| 26385/26385 [2:49:31<00:00,  2.59it/s]


All audio files generated and saved in audio_files_words. Updated file saved as parsed_cambridge_words_translated_with_audio.xlsx.


# Очистка бд

In [2]:
import sqlite3

# Подключаемся к базе данных
conn = sqlite3.connect('dictionary.db')
cursor = conn.cursor()

# Список таблиц, которые нужно очистить
tables = ['Levels', 'Translations', 'UserWordStatus', 'Words']

try:
    for table in tables:
        # Удаляем все записи из таблицы
        cursor.execute(f'DELETE FROM {table}')
        print(f'Очистка таблицы {table} завершена.')
        
        # Сбрасываем автоинкремент, если таблица содержит автоинкремент
        cursor.execute('DELETE FROM sqlite_sequence WHERE name = ?', (table,))
        print(f'Сброс автоинкремента для таблицы {table} завершен.')

    # Сохраняем изменения
    conn.commit()

except Exception as e:
    print(f'Произошла ошибка: {e}')

finally:
    # Закрываем соединение с базой данных
    cursor.close()
    conn.close()

print("База данных успешно очищена.")

Очистка таблицы Levels завершена.
Сброс автоинкремента для таблицы Levels завершен.
Очистка таблицы Translations завершена.
Сброс автоинкремента для таблицы Translations завершен.
Очистка таблицы UserWordStatus завершена.
Сброс автоинкремента для таблицы UserWordStatus завершен.
Очистка таблицы Words завершена.
Сброс автоинкремента для таблицы Words завершен.
База данных успешно очищена.


In [3]:
import sqlite3

# Подключаемся к базе данных
conn = sqlite3.connect('dictionary.db')
cursor = conn.cursor()

# Выполняем команду VACUUM
cursor.execute('VACUUM')
conn.commit()

# Закрываем соединение
conn.close()


# Запись в бд полностью (вместо с озвучкой)

In [5]:
import os
import pandas as pd
import sqlite3
import time
from tqdm import tqdm

# Путь к базе данных
db_path = 'dictionary.db'

# Загрузка данных из файла
file_path = 'parsed_cambridge_words_translated_with_audio.xlsx'
df = pd.read_excel(file_path)

# Подключение к базе данных
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Добавление нового столбца audio в таблицу Words (если он ещё не существует)
try:
    cursor.execute("ALTER TABLE Words ADD COLUMN audio BLOB")
    print("Столбец 'audio' был успешно добавлен.")
except sqlite3.OperationalError:
    print("Столбец 'audio' уже существует. Продолжаем обновление данных.")

# Функция для чтения аудиофайла в бинарном формате
def read_audio_file(file_path):
    try:
        with open(file_path, 'rb') as file:
            return file.read()
    except Exception as e:
        print(f"Ошибка при чтении файла {file_path}: {e}")
        return None

# Обновляем базу данных в один проход
for index, row in tqdm(df.iterrows(), total=len(df), desc="Обновление базы данных"):
    headword = row['headword']
    translation = row['translation']
    example = row['example']
    level = row['level']
    audio_path = row['audio_path']
    translated_example = row['translated_example'] if 'translated_example' in df.columns else None

    # Проверяем, есть ли перевод
    if pd.notna(translation) and translation.strip() != "":
        # Проверяем, существует ли уровень в таблице Levels
        cursor.execute("INSERT OR IGNORE INTO Levels (name) VALUES (?)", (level,))

        # Получаем id уровня
        cursor.execute("SELECT id FROM Levels WHERE name = ?", (level,))
        level_id = cursor.fetchone()[0]

        # Проверяем, существует ли headword в таблице Words
        cursor.execute("SELECT id FROM Words WHERE word = ?", (headword,))
        word_result = cursor.fetchone()

        if word_result is None:
            # Если headword нет, вставляем его в таблицу Words
            cursor.execute("INSERT INTO Words (word, level_id) VALUES (?, ?)", (headword, level_id))

            # Получаем id слова
            cursor.execute("SELECT id FROM Words WHERE word = ?", (headword,))
            word_id = cursor.fetchone()[0]
        else:
            # Если headword уже есть, получаем его id
            word_id = word_result[0]

        # Вставляем перевод и пример в таблицу Translations
        cursor.execute("INSERT INTO Translations (word_id, translation, usage_example, example_translation) VALUES (?, ?, ?, ?)", 
                       (word_id, translation, example, translated_example))

        # Добавляем аудиофайл, если он существует
        if not pd.isna(audio_path) and os.path.exists(audio_path):
            start_time = time.time()  # Замеряем время начала

            # Читаем аудиофайл и сохраняем его в BLOB
            audio_data = read_audio_file(audio_path)
            
            if audio_data:
                cursor.execute("UPDATE Words SET audio = ? WHERE id = ?", (audio_data, word_id))
                elapsed_time = time.time() - start_time  # Замеряем время выполнения
            else:
                print(f"Не удалось обновить аудио для слова '{headword}', файл: {audio_path}")
        else:
            print(f"Файл для аудио отсутствует или путь некорректен: {audio_path}")

# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

print("Данные и аудиофайлы успешно обновлены в базе данных.")


Столбец 'audio' уже существует. Продолжаем обновление данных.


Обновление базы данных:   2%|█▏                                                    | 593/26385 [00:59<43:06,  9.97it/s]


KeyboardInterrupt: 